In [ ]:
def sigmaM(M):
    sigmaM = 0.6082 +2.79064*M - 0.45345*M**2+0.02417*M**3-4.27*10**-4*M**4
    return sigmaM

def ng(M,z):
    dsdm=1/sigmaM(M)/np.log(10)*(2.79064-0.9069*M+0.07251*M**2-1.708*10**-3*M**3)
    a = 0.707
    A = 0.322184
    p = 0.3
    deltac = 1.686/cosmo.growthFactor(z)
    ng = -np.sqrt(2*a/np.pi)*A*(1+(a*deltac**2/sigmaM(M)**2)**-p)* \
    deltac/sigmaM(M)*np.exp(-a*deltac**2/2/sigmaM(M)**2)*dsdm
    return ng

def Rng(M,z,fnl):
    sm = sigmaM(M)
    deltac = 1.686/cosmo.growthFactor(z)
    S3 = fnl*10**(-2.25629-0.37109*M+0.02005*M**2)
    dsdm = -fnl*10**(-2.54966-0.34783*M+0.01944*M**2)
    ng = 1+sm**2/6/deltac*(S3*(deltac**4/sm**4-2*deltac**2/sm**2-1)+dsdm*(deltac**2/sm**2-1))
    #if fnl == 0:
    #   ng = 1
    return ng

def alpha(k):
    logk = np.log10(k)
    q = 0.75
    am = 10**(3.58603+17.33383*logk+14.68356*logk**2+6.08374*logk**3+\
            1.43971*logk**4+0.18336*logk**5+0.00977*logk**6)*q
    return am

def deltab(fnl, z, k):
    deltac = 1.686/cosmo.growthFactor(z)
    db = 2*fnl*deltac*alpha(k)
    return db

def b1(M,z):
    b = bias.haloBias(10**M, model = 'tinker10', z = z, mdef = 'vir')
    return b

def bias_eff(z,k,Mmin,fnl,bias_gauss=None):
    length = len(z)
    bu1 = np.zeros(length)
    bd1 = np.zeros(length)
    M = np.linspace(Mmin,18,100)
    dM = M[1:] - M[:-1]
    if bias_gauss != None:
        bias1 = bias_gauss
        bu1_ = np.zeros(length)
        bd1_ = np.zeros(length)
        for i in range(length):
            bu_kernel = np.zeros(length)
            bd_kernel = np.zeros(length)
            bu_kernel = (bias1+(bias1-1)*deltab(fnl,z[i],k[i]))*ng(M,z[i])*Rng(M,z[i],fnl)
            bd_kernel = ng(M,z[i])*Rng(M,z[i],fnl)
            hu = 0.5 * ( bu_kernel[1:] + bu_kernel[:-1] )
            hd = 0.5 * ( bd_kernel[1:] + bd_kernel[:-1] )
            bu1_[i] = np.sum(dM*hu)
            bd1_[i] = np.sum(dM*hd)
        b_eff = bu1_/bd1_
    else:
        for i in range(length):
            bu_kernel = np.zeros(length)
            bd_kernel = np.